## Import Libraries

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.LocalSession()

dependancy_path = 'dependencies.py'
bucket = sagemaker_session.default_bucket()
prefix = "Scikit-LinearLearner-pipeline-housing_data-example"

In [2]:
role = 'arn:aws:iam::600009802643:role/aws_sagemaker'

In [3]:
WORK_DIRECTORY = "."

train_input = sagemaker_session.upload_data(
    path="{}/{}".format(WORK_DIRECTORY, "housing_data.csv"),
    bucket=bucket,
    key_prefix="{}/{}".format(prefix, "train"),
)

arn:aws:iam::768014633529:role/aws-sagemaker-access


##  Create SageMaker Scikit Estimator 

In [4]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = 'sklearn_entry_point.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type='local',
    dependencies=[dependancy_path],
    sagemaker_session=sagemaker_session)

In [5]:
sklearn_preprocessor.fit({'train': train_data})

Creating tmpypb0cqvv_algo-1-oodg4_1 ... 
Attaching to tmpypb0cqvv_algo-1-oodg4_12mdone
algo-1-oodg4_1  | 2021-04-26 09:39:32,192 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
algo-1-oodg4_1  | 2021-04-26 09:39:32,200 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-oodg4_1  | 2021-04-26 09:39:32,242 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-oodg4_1  | 2021-04-26 09:39:35,181 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-oodg4_1  | 2021-04-26 09:39:35,252 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-oodg4_1  | 2021-04-26 09:39:35,290 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-oodg4_1  | 2021-04-26 09:39:35,319 sagemaker-training-toolkit INFO     Invoking user script
algo-1-oodg4_1  | 
algo-1-oodg4_1  | Training Env:
algo-1-ood

## Batch transform the training data

In [6]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='local',
    assemble_with = 'Line',
    accept = 'text/csv')

In [7]:
# Preprocess training input
transformer.transform(train_data, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

Attaching to tmp4w984g7f_algo-1-yaj84_1
algo-1-yaj84_1  | 2021-04-26 09:39:53,818 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-yaj84_1  | 2021-04-26 09:39:53,826 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-yaj84_1  | 2021-04-26 09:39:53,836 INFO - sagemaker-containers - nginx config: 
algo-1-yaj84_1  | worker_processes auto;
algo-1-yaj84_1  | daemon off;
algo-1-yaj84_1  | pid /tmp/nginx.pid;
algo-1-yaj84_1  | error_log  /dev/stderr;
algo-1-yaj84_1  | 
algo-1-yaj84_1  | worker_rlimit_nofile 4096;
algo-1-yaj84_1  | 
algo-1-yaj84_1  | events {
algo-1-yaj84_1  |   worker_connections 2048;
algo-1-yaj84_1  | }
algo-1-yaj84_1  | 
algo-1-yaj84_1  | http {
algo-1-yaj84_1  |   include /etc/nginx/mime.types;
algo-1-yaj84_1  |   default_type application/octet-stream;
algo-1-yaj84_1  |   access_log /dev/stdout combined;
algo-1-yaj84_1  | 
algo-1-yaj84_1  |   upstream gunicorn {
algo-1-yaj84_1  |     server unix:/tmp/guni

In [8]:
preprocessed_train

's3://sagemaker-us-east-2-768014633529/sagemaker-scikit-learn-2021-04-26-09-39-48-781'

## Real - Time Transformation of the training data

In [9]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime


timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix

#sm_model.sagemaker_session=None

scikit_learn_inference_model.deploy(initial_instance_count=1, instance_type='local', endpoint_name=endpoint_name)

Attaching to tmpsrp8uooy_algo-1-7irh1_1
algo-1-7irh1_1  | 2021-04-26 09:40:58,468 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-7irh1_1  | 2021-04-26 09:40:58,476 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-7irh1_1  | 2021-04-26 09:40:58,479 INFO - sagemaker-containers - nginx config: 
algo-1-7irh1_1  | worker_processes auto;
algo-1-7irh1_1  | daemon off;
algo-1-7irh1_1  | pid /tmp/nginx.pid;
algo-1-7irh1_1  | error_log  /dev/stderr;
algo-1-7irh1_1  | 
algo-1-7irh1_1  | worker_rlimit_nofile 4096;
algo-1-7irh1_1  | 
algo-1-7irh1_1  | events {
algo-1-7irh1_1  |   worker_connections 2048;
algo-1-7irh1_1  | }
algo-1-7irh1_1  | 
algo-1-7irh1_1  | http {
algo-1-7irh1_1  |   include /etc/nginx/mime.types;
algo-1-7irh1_1  |   default_type application/octet-stream;
algo-1-7irh1_1  |   access_log /dev/stdout combined;
algo-1-7irh1_1  | 
algo-1-7irh1_1  |   upstream gunicorn {
algo-1-7irh1_1  |     server unix:/tmp/guni

## Make a request to the endpoint

In [10]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, Predictor
#from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

csv_serializer = sagemaker.serializers.CSVSerializer(content_type="text/csv")
data = """-122.26,37.46,26,5067,750,1996,728,7.0001,NEAR OCEAN"""
#-118.15,34.2,52,1786,306,1018,322,4.1518,INLAND
#-118.39,34.12,29,6447,1012,2184,960,8.2816,<1H OCEAN


predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)


print(predictor.predict(data))

algo-1-7irh1_1  | 2021-04-26 09:41:10,447 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-7irh1_1  | ***********Model Function************
algo-1-7irh1_1  | ***********Predict Function************
b'{"instances": [{"features": [-1.3408764930120565, 0.8514634482951091, -0.21099315585118414, 1.1430410231864847, 0.5257730939190086, 0.5164773116405509, 0.6146760713962672, 1.6402561279658028, 0.5819441198401629, -0.030616763948385764, -1.0052566158906469, 0.0, 0.0, 0.0, 0.0, 1.0]}]}'algo-1-7irh1_1  | 172.18.0.1 - - [26/Apr/2021:09:41:11 +0000] "POST /invocations HTTP/1.1" 200 283 "-" "python-urllib3/1.26.4"

